In [34]:
RcaNotes={}
RcaNotes['c7ec380f50abcf86b04ee5d5b6896326b24fad38'] = 'Unrelated environment regresson'
RcaNotes['ce4bf92e46c8c73b33ced85f446f7ac5daecd4f5'] = '[2500] [Jake] Headers optimization'
RcaNotes['1ef6e399f114a0fd580272d4cdca86b9f8732cf3'] = 'Unrelated environment recovery from earlier regresson'
RcaNotes['29e81b578c876b1686e532f5a63d229e2b48f327'] = '''3.13.1 Direct pacakge: '''
RcaNotes['ed0c8f0ed7f084476321e54ffb7ec6be0b83b0b2'] = 'Unrelated environment gain'
RcaNotes['6de15df3f5320eb648df754b2d9f02d9fd2e654b'] = '''[1420] 3.13.2 Direct package 
 <br> [600] [Jake] TransportSerialization optimization
 <br> [400] [Jake] SessionTokenMismatchRetryPolicy optimization
 <br>      [Vinod] RntbdReqeustPooling
 <br>      [Vinod] RntbdResponsePooling
 <br>      [Vinod] RespinseSerializationAsync
'''
RcaNotes['5618b95c787e274b865ad6bccf14261bb0b880cb'] = '[389] [Jake] VectorSessionTokenOptimization'

RcaNotes

In [ ]:
%%sql --database testdb --container runsummary --output runlogs

select 
    c.Commit AS Commit,
    CONCAT(c.Date, ":", c.Time) AS RunTime,
	c.Top10PercentAverageRps AS Top10AVGRps
from c 
where c.Cores = 4 
    and c.TotalOps >= 500000
    and c.Top10PercentAverageRps > 1000
    and CONCAT(c.Date, ":", c.Time) > "2020-08-00:20-15"
    and CONCAT(c.Date, ":", c.Time) < "2020-09-28:01-00"
    and c.BranchName = "master"
    and c.WorkloadType = "ReadStreamExistsV3"


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pylab as plt

df=pd.DataFrame(runlogs.groupby('Commit', as_index=False).agg({'RunTime': ['min'], 'Top10AVGRps': ['mean']}))

df.columns=df.columns.to_flat_index()
df.sort_values(('RunTime', 'min'), inplace=True)

df['RunTimeMin']=df[('RunTime', 'min')]
df['Top10AVGRpsMean']=df[('Top10AVGRps', 'mean')]
df['Commit']=df[('Commit', '')]

commitNotes = {}
for index, row in df.iterrows():
    commitNotes[index] = ''
    if row['Commit'] in RcaNotes:
        commitNotes[index] = RcaNotes[row['Commit']]
        
df['RcaNotes']=pd.Series(commitNotes)
df['Top10AVGRpsMeanDiff']=round(df['Top10AVGRpsMean'].copy().pct_change() * 100, 2)
df.drop(columns=[('Top10AVGRps', 'mean'), ('RunTime', 'min'), ('Commit', '')], inplace=True)

figVertical = go.Figure(go.Waterfall(
    x = df['RunTimeMin'],
    text = '<br> Commit:' + df['Commit'] +  '<br> Diff: ' + df['Top10AVGRpsMeanDiff'].astype(str) + ' <br><br> ' + df['RcaNotes'] + ' <br> ',
    y = df['Top10AVGRpsMeanDiff'],
    connector = {'line':{'color':'rgb(63, 63, 63)'}},
))

figVertical.update_layout(
    title='ReadStreamExistsV3 Benchmark: Percentage of impact',
    yaxis_title='Percentage',
    xaxis_title='Ordered commit time-line',
    #showlegend = True,
    height = 700,
    font=dict(
        family='Courier New, monospace',
        size=18,
        color='RebeccaPurple'
    )    
)

figVertical.show()

figHorizontal = go.Figure(go.Waterfall(
    orientation = 'h',
    y = df['RunTimeMin'],
    text = '<br> Commit:' + df['Commit'] +  '<br> Diff: ' + df['Top10AVGRpsMeanDiff'].astype(str) + ' <br><br> ' + df['RcaNotes'] + ' <br> ',
    x = df['Top10AVGRpsMeanDiff'],
    connector = {'line':{'color':'rgb(63, 63, 63)'}},
))

figHorizontal.update_layout(
    title='ReadStreamExistsV3 Benchmark: Percentage of impact',
    xaxis_title='Percentage',
    yaxis_title='Ordered commit time-line',
    #showlegend = True,
    height = 700,
    font=dict(
        family='Courier New, monospace',
        size=18,
        color='RebeccaPurple'
    )    
)
figHorizontal.show()

In [ ]:
import plotly.express as px
import pandas as pd
import matplotlib.pylab as plt

df=pd.DataFrame(runlogs.groupby("Commit", as_index=False).agg({'RunTime': ['min'], 'Top10AVGRps': ['mean']}))

df.columns=df.columns.to_flat_index()
df.sort_values(('RunTime', 'min'), inplace=True)

df["RunTimeMin"]=df[('RunTime', 'min')]
df["Top10AVGRpsMean"]=df[('Top10AVGRps', 'mean')]
df["Commit"]=df[('Commit', '')]

commitNotes = {}
for index, row in df.iterrows():
    commitNotes[index] = ''
    if row['Commit'] in RcaNotes:
        commitNotes[index] = RcaNotes[row['Commit']]
        
df["RcaNotes"]=pd.Series(commitNotes)
df["Top10AVGRpsMeanDiff"]=df["Top10AVGRpsMean"].copy().diff()
df.drop(columns=[('Top10AVGRps', 'mean'), ('RunTime', 'min'), ('Commit', '')], inplace=True)

fig = px.scatter(df, 
               x='RunTimeMin', 
               y='Top10AVGRpsMean', 
               title="ReadStreamExistsV3 RPS F4vCore Linux (.NET) - Per commit", 
               hover_data=['RunTimeMin', 'Top10AVGRpsMean', 'Commit', 'RcaNotes'], 
               #color='ImpactType',
               height=800)
    
fig.update_traces(mode="markers+lines")
fig.show()

In [ ]:
import pandas as pd
import matplotlib.pylab as plt

tmpCommit=runlogs

tmpCommit['RunTime']=pd.to_datetime(tmpCommit['RunTime'], format="%Y-%m-%d:%H-%M")
tmpCommit.set_index('Commit', inplace=True)

fig, ax = plt.subplots(figsize=(30,10))
tmpCommit.groupby("Commit").plot(x='RunTime', y='Top10AVGRps', ax=ax)
plt.legend([v[0] for v in tmpCommit.groupby('Commit')])
ax.get_legend().remove()